<a href="https://www.kaggle.com/code/mohankumarmanepalli/oct-3classes?scriptVersionId=164433039" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install keract

In [2]:
import os
from glob import glob
import pandas as pd
import numpy as np
from numpy import expand_dims
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sn
from skimage.transform import resize
from skimage.color import gray2rgb
from sklearn.metrics import classification_report, confusion_matrix
from IPython.display import SVG
import keract
from tensorflow import keras
from tensorflow.keras import applications, optimizers
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.utils import to_categorical, model_to_dot, plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau

2024-02-26 19:04:24.534511: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 19:04:24.534603: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 19:04:24.679080: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
img_width, img_height = 150, 150 
channels = 3
batch_size = 32

In [4]:
train_path="/kaggle/input/oct-classes/OCT3Classes/OCT_3_Classes/OCT2017 /train"
val_path="/kaggle/input/oct-classes/OCT3Classes/OCT_3_Classes/OCT2017 /val"
test_path="/kaggle/input/oct-classes/OCT3Classes/OCT_3_Classes/OCT2017 /test"
train_datagen = ImageDataGenerator(
    rescale= 1./255,
    zoom_range= (0.73, 0.9),
    horizontal_flip= True,
    rotation_range= 10,
    width_shift_range= 0.10,
    fill_mode= 'constant',
    height_shift_range= 0.10,   
    brightness_range= (0.55, 0.9),
)
val_datagen = ImageDataGenerator(
    rescale= 1./255, 
)

train_dataset=train_datagen.flow_from_directory(  
    train_path,  
    target_size= (150, 150), 
    color_mode= 'rgb',
    batch_size= 32,  
    class_mode= 'categorical',
    shuffle= True, 
    seed= 1337
)
val_dataset = val_datagen.flow_from_directory(
    val_path,
    target_size= (150, 150),
    color_mode= 'rgb',
    batch_size= 32,  
    class_mode= 'categorical',
    shuffle= True, 
    seed= 1337
)
test_dataset = val_datagen.flow_from_directory(  
    test_path,  
    target_size= (150, 150), 
    color_mode= 'rgb',
    batch_size= 32,        
    class_mode= 'categorical',
    shuffle= False, 
)

Found 59208 images belonging to 3 classes.
Found 24 images belonging to 3 classes.
Found 726 images belonging to 3 classes.


In [5]:
num_classes = len(train_dataset.class_indices)  
train_labels = train_dataset.classes 
train_labels = to_categorical(train_labels, num_classes=num_classes)
valid_labels = val_dataset.classes 
valid_labels = to_categorical(valid_labels, num_classes=num_classes)
nb_train_samples = len(train_dataset.filenames)  
nb_valid_samples = len(val_dataset.filenames)
nb_test_samples = len(test_dataset.filenames)

In [6]:
vgg16 = VGG16(include_top= False, input_shape= (img_width, img_height, channels), weights= 'imagenet')
vgg16.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                              

In [7]:
model = Sequential()

for layer in vgg16.layers:
    model.add(layer)

for layer in model.layers:
    layer.trainable= False

model.add(Flatten(input_shape= (4, 4, 512)))
model.add(Dropout(0.2))
model.add(Dense(3,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 37, 37, 256)       2

In [8]:
model.compile(optimizer= keras.optimizers.Adam(lr= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

In [9]:
checkpoint = ModelCheckpoint(
    'baseline_model.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='auto',
    save_weights_only=False,
    period=1
)

earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    verbose=1,
    mode='auto'
)

csvlogger = CSVLogger(
    filename= "baseline_training_csv.log",
    separator = ",",
    append = False
)

reduceLR = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,
    verbose=1, 
    mode='auto'
)

callbacks = [checkpoint, earlystop, csvlogger,reduceLR]

In [10]:
history = model.fit(
    train_dataset, 
    epochs = 10,
    steps_per_epoch = nb_train_samples//batch_size,
    validation_data = val_dataset, 
    validation_steps = 1,
    verbose = 2,
    callbacks = callbacks,
    shuffle = True
)

Epoch 1/10


I0000 00:00:1708974335.608841      92 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Epoch 1: val_loss improved from inf to 2.51014, saving model to baseline_model.h5
1850/1850 - 743s - loss: 0.4981 - accuracy: 0.8076 - val_loss: 2.5101 - val_accuracy: 0.5833 - lr: 0.0010 - 743s/epoch - 401ms/step
Epoch 2/10


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.51014
1850/1850 - 495s - loss: 0.4507 - accuracy: 0.8301 - val_loss: 2.7913 - val_accuracy: 0.5833 - lr: 0.0010 - 495s/epoch - 268ms/step
Epoch 3/10

Epoch 3: val_loss improved from 2.51014 to 2.46755, saving model to baseline_model.h5
1850/1850 - 474s - loss: 0.4448 - accuracy: 0.8321 - val_loss: 2.4675 - val_accuracy: 0.5417 - lr: 0.0010 - 474s/epoch - 256ms/step
Epoch 4/10

Epoch 4: val_loss did not improve from 2.46755
1850/1850 - 478s - loss: 0.4387 - accuracy: 0.8364 - val_loss: 2.9858 - val_accuracy: 0.5417 - lr: 0.0010 - 478s/epoch - 258ms/step
Epoch 5/10

Epoch 5: val_loss improved from 2.46755 to 2.19215, saving model to baseline_model.h5
1850/1850 - 477s - loss: 0.4369 - accuracy: 0.8369 - val_loss: 2.1921 - val_accuracy: 0.5833 - lr: 0.0010 - 477s/epoch - 258ms/step
Epoch 6/10

Epoch 6: val_loss did not improve from 2.19215
1850/1850 - 502s - loss: 0.4392 - accuracy: 0.8357 - val_loss: 3.2074 - val_accuracy: 0.6250 - lr: 0.0010 - 50

In [11]:
(eval_loss, eval_accuracy) = model.evaluate(test_dataset, batch_size = batch_size, verbose= 1)
print('Test Loss: ', eval_loss)
print('Test Accuracy: ', eval_accuracy)

23/23 [==============================] - 8s 358ms/step - loss: 0.3586 - accuracy: 0.8939
Test Loss:  0.35858550667762756
Test Accuracy:  0.8939393758773804


In [12]:
model.trainable= True
model.compile(optimizer= keras.optimizers.Adam(1e-5), loss= 'categorical_crossentropy', metrics= ['accuracy'])

In [13]:
checkpoint = ModelCheckpoint(
    'finetuned_model.h5',
    monitor='loss',
    verbose=1,
    save_best_only=True,
    mode='auto',
    save_weights_only=False,
    period=1
)

earlystop = EarlyStopping(
    monitor='loss',
    min_delta=0.001,
    patience=3,
    verbose=1,
    mode='auto'
)

csvlogger = CSVLogger(
    filename= "finetuned_training_csv.log",
    separator = ",",
    append = False
)

reduceLR = ReduceLROnPlateau(
    monitor='loss',
    factor=0.1,
    patience=3,
    verbose=1, 
    mode='auto'
)

callbacks = [checkpoint, earlystop, csvlogger,reduceLR]

In [14]:
history_1 = model.fit(
    train_dataset, 
    epochs = 10,
    steps_per_epoch = nb_train_samples//batch_size,
    validation_data = val_dataset, 
    validation_steps = 1,
    verbose = 2,
    callbacks = callbacks,
    shuffle = True
)

Epoch 1/10

Epoch 1: loss improved from inf to 0.27751, saving model to finetuned_model.h5
1850/1850 - 547s - loss: 0.2775 - accuracy: 0.9025 - val_loss: 1.0033 - val_accuracy: 0.7500 - lr: 1.0000e-05 - 547s/epoch - 296ms/step
Epoch 2/10

Epoch 2: loss improved from 0.27751 to 0.18722, saving model to finetuned_model.h5
1850/1850 - 540s - loss: 0.1872 - accuracy: 0.9364 - val_loss: 0.7246 - val_accuracy: 0.6667 - lr: 1.0000e-05 - 540s/epoch - 292ms/step
Epoch 3/10

Epoch 3: loss improved from 0.18722 to 0.15915, saving model to finetuned_model.h5
1850/1850 - 538s - loss: 0.1592 - accuracy: 0.9462 - val_loss: 0.6437 - val_accuracy: 0.7917 - lr: 1.0000e-05 - 538s/epoch - 291ms/step
Epoch 4/10

Epoch 4: loss improved from 0.15915 to 0.14617, saving model to finetuned_model.h5
1850/1850 - 547s - loss: 0.1462 - accuracy: 0.9498 - val_loss: 0.1840 - val_accuracy: 0.9167 - lr: 1.0000e-05 - 547s/epoch - 296ms/step
Epoch 5/10

Epoch 5: loss improved from 0.14617 to 0.13182, saving model to fine

In [15]:
(eval_loss, eval_accuracy) = model.evaluate(test_dataset, batch_size= batch_size, verbose= 1)
print('Test Loss: ', eval_loss)
print('Test Accuracy: ', eval_accuracy)

23/23 [==============================] - 3s 124ms/step - loss: 0.0132 - accuracy: 1.0000
Test Loss:  0.013199358247220516
Test Accuracy:  1.0


In [16]:
model.save_weights("Classes_3_Model.h5")